# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# Reduced Data: Grid Search + Cross-Validation

This time, as explored in the EDA NB, let's consider removing customers who we feel have too few or too many purchases to influence the model in the intended way.

Examples:
- Too few - Customers who have only bought 1 comic (series).
- Too many - Customers with > 1000 series (for example, think all eBay customers are rolled into one account number).

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
# %autoreload 1 #would be where you need to specify the files
# %aimport comic_recs

import pandas as pd # dataframes
import os
import pickle
import numpy as np

# Data storage
from sqlalchemy import create_engine # SQL helper
#import psycopg2 as psql #PostgreSQL DBs

# import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql.types import (StructType, StructField, IntegerType
#                                ,FloatType, LongType, StringType)
from pyspark.sql.types import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, lit, isnan, when, count
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import (CrossValidator, ParamGridBuilder, 
                               TrainValidationSplit)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import DataFrame

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import sys

In [3]:
sys.path.append('..')

In [4]:
# Custom
import data_fcns as dfc
import keys  # Custom keys lib
import comic_recs as cr

import time
import itertools
from functools import reduce
import numpy as np

In [5]:
from pyspark import SparkConf

conf = SparkConf()

conf = (conf.setMaster('local[*]')
#         .set('spark.executor.memory', '1G') #https://stackoverflow.com/questions/48523629/spark-pyspark-an-error-occurred-while-trying-to-connect-to-the-java-server-127
        .set('spark.driver.memory', '7G')
        .set('spark.driver.maxResultSize', '2G'))
#         .set('spark.executor.memory', '1G')
#         .set('spark.driver.memory', '10G')
#         .set('spark.driver.maxResultSize', '5G'))

sc = pyspark.SparkContext().getOrCreate(conf=conf)

from pyspark.sql import SQLContext
sql_context = SQLContext(sc)

sc.setCheckpointDir('./checkpoints')

# spark.sparkContext.setCheckpointDir("hdfs://datalake/check_point_directory/als")

In [6]:
# # spark config
# spark = pyspark.sql.SparkSession \
#     .builder \
#     .appName("comic recs") \
#     .config("spark.driver.maxResultSize", "8g") \
#     .config("spark.driver.memory", "8g") \
#     .config("spark.executor.memory", "8g") \
#     .config("spark.master", "local[*]") \
#     .getOrCreate()

# instantiate SparkSession object
spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local").getOrCreate()

# spark config
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("movie recommendation") \
    .config("spark.driver.maxResultSize", "1g") \
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "20g") \
    .config("spark.master", "local[*]") \
    .getOrCreate()

## Import Data

We've previously set aside the dataset into a `json` file.

In [7]:
# pickleRdd = sc.pickleFile('raw_data/als_input_filtered_190915.pkl').collect()
# sold = sql_context.createDataFrame(pickleRdd)

In [8]:
# We have previously created a version of the transactions table 
# and filtered it down.
sold = sql_context.read.json('raw_data/als_input_filtered_190915.json')

In [9]:
# Persist the data
sold.persist()

DataFrame[account_id: bigint, bought: bigint, comic_id: bigint]

In [10]:
sold.count()

846090

### ALS Model

Let's start with  train/test split.

In [11]:
random_seed = 1234

In [12]:
# Split data into training and test set
(train, test) = sold.randomSplit([.75, .25], seed=random_seed)

In [13]:
train.count()

634446

Make sure shapes make sense.

In [14]:
print(train.count(), len(train.columns))

634446 3


In [15]:
print(test.count(), len(test.columns))

211644 3


## New Grid Search

In [17]:
# hyper-param config
max_iters = [20, 25, 30]
ranks = [5, 10, 20, 25]
reg_params = [.05, .10, .15]
alphas = [5, 25, 40]

In [25]:
# hyper-param config
max_iters = [30, 40]
ranks = [25, 30]
reg_params = [.15, .25]
alphas = [5]

In [42]:
# hyper-param config
max_iters = [30]
ranks = [5, 10]
reg_params = [.25, .30]
alphas = [5]

In [51]:
# hyper-param config
max_iters = [40]
ranks = [30]
reg_params = [.25]
alphas = [5]

In [55]:
# Instantiate list to hold candidate models
model_list = []

In [56]:
# For loop will automatically create and store ALS models
for r in ranks:
    for mi in max_iters:
        for rp in reg_params:
            for a in alphas:
                model_list.append(ALS(userCol= "account_id"
                                      ,itemCol= "comic_id"
                                      ,ratingCol= "bought"
                                      ,rank = r, maxIter = mi, regParam = rp
                                      ,alpha = a
                                      ,coldStartStrategy="drop"
                                      ,nonnegative = True
                                      ,implicitPrefs = True))


In [57]:
len(model_list)

1

In [58]:
len(max_iters)*len(ranks)*len(reg_params)*len(alphas)

1

### Manual Cross-Validation

In [70]:
#Building 5 folds within the training set.
train1, train2, train3, train4, train5 = (train.randomSplit(
                                         [0.2, 0.2, 0.2, 0.2, 0.2], seed = 1234))
fold1 = train2.union(train3).union(train4).union(train5)
fold2 = train3.union(train4).union(train5).union(train1)
fold3 = train4.union(train5).union(train1).union(train2)
fold4 = train5.union(train1).union(train2).union(train3)
fold5 = train1.union(train2).union(train3).union(train4)

In [71]:
foldlist = [(fold1, train1), (fold2, train2), (fold3, train3)
            , (fold4, train4), (fold5, train5)]


## Metrics

In [30]:
# This function was initially created for the Million Songs Echo Nest Taste Profile dataset which had 3 columns: userId, songId,
# and num_plays. The column num_plays was used as implicit ratings with the ALS algorithm.

def ROEM(predictions):
    #Creates predictions table that can be queried
    predictions.createOrReplaceTempView("predictions") 

    #Sum of total number of plays of all songs
    denominator = predictions.groupBy().sum("bought").collect()[0][0]

    #Calculating rankings of songs predictions by user
    #spark.sql("SELECT account_id, bought, PERCENT_RANK() OVER (PARTITION BY account_id ORDER BY prediction DESC) AS rank FROM predictions").createOrReplaceTempView("rankings")
    sql_context.sql("SELECT account_id, bought, PERCENT_RANK() OVER (PARTITION BY account_id ORDER BY prediction DESC) AS rank FROM predictions").createOrReplaceTempView("rankings")

    #Multiplies the rank of each song by the number of plays for each user
    #and adds the products together
    #  numerator = spark.sql('SELECT SUM(bought * rank) FROM rankings').collect()[0][0]
    numerator = sql_context.sql('SELECT SUM(bought * rank) FROM rankings').collect()[0][0]

    return numerator / denominator

In [ ]:
model_list_test = model_list[:1]

In [ ]:
model_list_test

In [ ]:
# Empty list to fill with ROEMs from each model
ROEMS = []

In [47]:
ROEMS_new = []

In [59]:
# Loops through all models and all folds
for model in model_list:
#     for ft_pair in foldlist:

#         # Fits model to fold within training data
#         fitted_model = model.fit(ft_pair[0])

#         # Generates predictions using fitted_model on respective CV test data
#         predictions = fitted_model.transform(ft_pair[1])

#         # Generates and prints a ROEM metric CV test data
#         r = ROEM(predictions)
#         print ("ROEM: ", r)

    # Fits model to all of training data and generates preds for test data
    v_fitted_model = model.fit(train)
    v_predictions = v_fitted_model.transform(test)
    v_ROEM = ROEM(v_predictions)

    # Adds validation ROEM to ROEM list
    ROEMS_new.append(v_ROEM)
    print ("Validation ROEM: ", v_ROEM)

Validation ROEM:  0.15106942086275174


In [60]:
len(ROEMS_new)

5

In [63]:
# Find the index of the smallest ROEM
idx = np.argmin(ROEMS_new)
print("Index of smallest ROEM:", idx)

# Find ith element of ROEMS
print("Smallest ROEM: ", ROEMS_new[idx])

Index of smallest ROEM: 4
Smallest ROEM:  0.15106942086275174


In [62]:
# Find the index of the smallest ROEM
idx = np.argmin(ROEMS_new)
print("Index of smallest ROEM:", idx)

# Find ith element of ROEMS
print("Smallest ROEM: ", ROEMS_new[idx])

Index of smallest ROEM: 4
Smallest ROEM:  0.15106942086275174


## Extracting Parameters

In [66]:
# Extract the best_model
best_model = model_list[0]

print('alpha: {}'.format(best_model.getAlpha()))
print('reg param: {}'.format(best_model.getRegParam()))
print('max iter: {}'.format(best_model.getMaxIter()))
print('rank: {}'.format(best_model.getRank()))

alpha: 5.0
reg param: 0.25
max iter: 40
rank: 30


In [67]:
best_model.save('models/best_model_use')

In [40]:
# with open('support_data/best_model_20190916a.pkl', 'wb') as f:
#     pickle.dump(best_model, f)
    
# # Example - load pickle
# # pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# # pe1 = pickle.load(pickle_in)

#### Use this to reload the Grid Search results

In [41]:
# pickle_in = open('support_data/params_errs_20190912d.pkl', 'rb')
# params_errs_4 = pickle.load(pickle_in)
                         

## Cross-Validation

In [68]:
errors = []

In [73]:
for ft_pair in foldlist:

    # Fits model to fold within training data
    fitted_model = best_model.fit(ft_pair[0])

    # Generates predictions using fitted_model on respective CV test data
    predictions = fitted_model.transform(ft_pair[1])

    # Generates and prints a ROEM metric CV test data
    r = ROEM(predictions)
    errors.append(r)


In [74]:
print("CV errors range: %0.2f (+/- %0.2f)" % (np.mean(errors), np.std(errors) * 2))

CV errors range: 0.16 (+/- 0.00)


In [83]:
errors = errors[-5:]

In [84]:
errors

[0.16524238417949952,
 0.1663290924381311,
 0.16134557238452693,
 0.16070017898764644,
 0.1645815844959141]

## Test the Candidate Model

Test vs our holdout set.

In [89]:
model_fitted = best_model.fit(train)

In [90]:
# get predictions on test
test_preds = model_fitted.transform(test)

# Evaluate test
test_roem = ROEM(test_preds)
test_roem

0.15106942086275174

Weird that test error is lower than train...but not too far off. Let's run with it for now.

In [93]:
best_model.getAlpha()

5.0

In [94]:
best_model.getRank()

30

## Get Factors

#### Save the item factors for future use!

In [91]:
item_factors = model_fitted.itemFactors.toPandas()

In [92]:
item_factors.shape

(790, 2)

In [95]:
item_factors.to_pickle("support_data/item_factors_20190916.pkl")

In [96]:
pd.set_option('display.max_colwidth', -1)

In [97]:
item_factors.head()

,id,features
0,60,"[0.01743680238723755, 0.007148396223783493, 1.06563401222229, 0.04059157520532608, 0.019294701516628265, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006994778756052256, 0.0, 0.0, 0.0, 0.017005732282996178, 0.0, 0.036391135305166245, 0.0, 0.0, 0.0, 0.2501244843006134, 0.0, 0.0, 0.0, 0.0]"
1,80,"[0.0, 0.0, 0.03249182552099228, 0.11323057115077972, 0.0, 0.0, 0.0, 0.7948412299156189, 0.0, 0.0, 0.0, 0.0, 0.23714490234851837, 0.03495769575238228, 0.29481297731399536, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22936636209487915, 0.03450985625386238, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12904486060142517]"
2,110,"[0.3528019189834595, 0.0, 0.0, 0.0, 0.2717379629611969, 0.09929965436458588, 0.03336532041430473, 0.0, 0.0, 0.02282828837633133, 0.00937669724225998, 0.03452523052692413, 0.0, 0.0, 0.11529584974050522, 0.0, 0.0, 0.0, 0.12241517752408981, 0.01565462350845337, 0.13469548523426056, 0.003043871372938156, 0.856399416923523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12838362157344818, 0.10266077518463135]"
3,200,"[0.014895367436110973, 0.32599085569381714, 0.09774906188249588, 0.05869003385305405, 0.016703536733984947, 0.0, 0.13506878912448883, 0.0, 0.0, 0.27643218636512756, 0.0, 0.03294535353779793, 0.0, 0.0, 0.18430395424365997, 0.9149331450462341, 0.13594630360603333, 0.0, 0.00893563311547041, 0.0, 0.0, 0.013972891494631767, 0.0, 0.0011329196859151125, 0.03825383633375168, 0.08892543613910675, 0.05580912530422211, 0.0, 0.0, 0.4417659640312195]"
4,240,"[0.0, 0.05928525701165199, 0.0, 0.0, 0.7101511359214783, 0.0, 0.08753978461027145, 0.0, 0.06151632219552994, 0.2771930396556854, 0.0, 0.007922252640128136, 0.0, 0.0, 0.3503260612487793, 0.14580319821834564, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11093205213546753, 0.19919617474079132, 0.0, 0.3611432611942291, 0.0]"


Test unpickle

In [98]:
unpickled_items = pd.read_pickle('support_data/item_factors_20190916.pkl')

In [27]:
def get_cv_errors(folds, als, evaluator):
    """
    Given dictionary of spark DF folds and an ALS object
    returns list of errors
    Parameters
    ----------
    folds = list of spark dataframes
    als = ALS instance
    evaluator = spark Evaluator instance, usually regression

    Returns
    -------
    list of each test fold's prediction error metric
    """
    errors = []

    for i in range(len(folds)):

        # Partition out train and test
        test_fold_df = folds[i]

        train_folds = list(set(folds) - set([test_fold_df]))
        train_fold_df = reduce(DataFrame.unionAll, train_folds)

        # fit on train
        model = als.fit(train_fold_df)

        # get predictions on test
        preds = model.transform(test_fold_df)

        # Evaluate test
        errors.append(evaluator.evaluate(preds))

        # done
    return errors

In [ ]:
# Evaluate the model by computing the RMSE on the test data
eval_reg = RegressionEvaluator(metricName="rmse"
                               , labelCol="bought"
                               , predictionCol="prediction")

### Grid Search

Let's further subset into test and validation sets.

In [ ]:
# Split data into training and validation sets
(gs_train, gs_val) = train.randomSplit([(1-(1/3)), (1/3)], seed=random_seed)

In [ ]:
print(gs_train.count(), len(gs_train.columns))

In [ ]:
print(gs_val.count(), len(gs_val.columns))

In [ ]:
# hyper-param config
num_iterations = [5, 10, 20]
ranks = [5, 10, 20]
reg_params = [0.01, 0.1, 1]
alphas = [5, 25, 40]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

Save the descriptive results

In [ ]:
param_errs_rd_1 = params_errs

In [ ]:
with open('support_data/params_errs_20190912a.pkl', 'wb') as f:
    pickle.dump(param_errs_rd_1, f)
    
# Example - load pickle
# pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# pe1 = pickle.load(pickle_in)

#### Use this to reload the Grid Search results

In [ ]:
pickle_in = open('support_data/params_errs_20190912a.pkl', 'rb')
params_errs_1 = pickle.load(pickle_in)
                         

Hmmm. Let's put `params_errs_1` into a dataframe and find the model with the lowest error!

In [ ]:
gs_cols = ['max_iters', 'reg', 'rank', 'alpha', 'rmse']

In [ ]:
gs_df = pd.DataFrame(params_errs_1, columns=gs_cols)

In [ ]:
gs_df.head()

In [ ]:
min_err = gs_df.rmse.min()

In [ ]:
min_df = gs_df.loc[gs_df['rmse']==min_err]

In [ ]:
min_df

In [ ]:
best_max_iter = min_df['max_iters'].iloc[0]
best_reg = min_df['reg'].iloc[0]
best_rank = min_df['rank'].iloc[0]
best_alpha = min_df['alpha'].iloc[0]

Let's do some visual comparisons.

In [ ]:
gs_rank_match = (gs_df['rank']==best_rank)
gs_reg_match = (gs_df['reg']==best_reg)
gs_iter_match = (gs_df['max_iters']==best_max_iter)
gs_alpha_match = (gs_df['alpha']==best_alpha)

In [ ]:
gs_vary_rank = gs_df.loc[(gs_reg_match & gs_iter_match & gs_alpha_match),:]

In [ ]:
gs_vary_rank

In [ ]:
gs_vary_alpha = gs_df.loc[(gs_reg_match & gs_iter_match & gs_rank_match),:]

In [ ]:
gs_vary_alpha

In [ ]:
gs_vary_reg = gs_df.loc[(gs_alpha_match & gs_iter_match & gs_rank_match),:]

In [ ]:
gs_vary_reg

In [ ]:
gs_vary_iter = gs_df.loc[(gs_alpha_match & gs_reg_match & gs_rank_match),:]

In [ ]:
gs_vary_iter

So quick inspection on these, lets:
- Keep `rank` = 5  
- Stick with regularization parameter at `0.01`.  
- Max iterations it seems will plateau, but let's investigate going up to maybe 30 or so.
- Looking like going higher on alpha might lower RMSE more.


So let's go through another grid of parameters.

In [ ]:
# hyper-param config
num_iterations = [20, 30, 40]
ranks = [5]
reg_params = [0.01]
alphas = [40, 50, 100]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs_2 = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
with open('support_data/params_errs_20190912b.pkl', 'wb') as f:
    pickle.dump(params_errs_2, f)
    
# Example - load pickle
# pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# pe1 = pickle.load(pickle_in)

#### Use this to reload the Grid Search results

In [ ]:
pickle_in = open('support_data/params_errs_20190912b.pkl', 'rb')
params_errs_2 = pickle.load(pickle_in)
                         

Increasing iterations doesn't seem t materially improve RMSE, but increasing alpha does. So let's try a large range of alphas.


In [ ]:
num_iterations = [20]
alphas = [50, 100, 500, 1000, 1500, 2000]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs_3 = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
with open('support_data/params_errs_20190912c.pkl', 'wb') as f:
    pickle.dump(params_errs_3, f)
    
# Example - load pickle
# pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# pe1 = pickle.load(pickle_in)

#### Use this to reload the Grid Search results

In [ ]:
pickle_in = open('support_data/params_errs_20190912c.pkl', 'rb')
params_errs_3 = pickle.load(pickle_in)
                         

So it appears that `alpha` = 1000 is where min RMSE is so far. For due diligence let's look at fewer latent factors.

In [ ]:
ranks = [2,3,4,5,10,20,30,50]
alphas = [1000]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs_4 = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
with open('support_data/params_errs_20190912d.pkl', 'wb') as f:
    pickle.dump(params_errs_4, f)
    
# Example - load pickle
# pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# pe1 = pickle.load(pickle_in)

#### Use this to reload the Grid Search results

In [ ]:
pickle_in = open('support_data/params_errs_20190912d.pkl', 'rb')
params_errs_4 = pickle.load(pickle_in)
                         

Looks like 3 is the number of latent factors that minimizes RMSE. 

Let's put together a final run grid that we can use to compare different hyperparameter combinations.

In [ ]:
# hyper-param config
num_iterations = [10, 20, 30]
ranks = [3,5,10]
reg_params = [0.01, 0.1,0.5]
alphas = [40, 50, 100, 1000, 2000]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs_5 = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
with open('support_data/params_errs_20190912e.pkl', 'wb') as f:
    pickle.dump(params_errs_5, f)
    
# Example - load pickle
# pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# pe1 = pickle.load(pickle_in)

#### Use this to reload the Grid Search results

In [ ]:
pickle_in = open('support_data/params_errs_20190912e.pkl', 'rb')
params_errs_5 = pickle.load(pickle_in)
                         

In [ ]:
# hyper-param config
num_iterations = [20]
ranks = [3]
reg_params = [0.01, 0.1,0.5,0.7,1]
alphas = [1000]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs_x = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
d = ALS()

In [ ]:
gs_cols = ['max_iters', 'reg', 'rank', 'alpha', 'rmse']

In [ ]:
gs_df = pd.DataFrame(params_errs_5, columns=gs_cols)

In [ ]:
gs_df.head()

In [ ]:
min_err = gs_df.rmse.min()

In [ ]:
min_df = gs_df.loc[gs_df['rmse']==min_err]

In [ ]:
min_df

In [ ]:
best_max_iter = min_df['max_iters'].iloc[0]
best_reg = min_df['reg'].iloc[0]
best_rank = min_df['rank'].iloc[0]
best_alpha = min_df['alpha'].iloc[0]

Let's do some visual comparisons.

In [ ]:
gs_rank_match = (gs_df['rank']==best_rank)
gs_reg_match = (gs_df['reg']==best_reg)
gs_iter_match = (gs_df['max_iters']==best_max_iter)
gs_alpha_match = (gs_df['alpha']==best_alpha)

In [ ]:
gs_vary_rank = gs_df.loc[(gs_reg_match & gs_iter_match & gs_alpha_match),:]

In [ ]:
gs_vary_rank

In [ ]:
gs_vary_alpha = gs_df.loc[(gs_reg_match & gs_iter_match & gs_rank_match),:]

In [ ]:
gs_vary_alpha

In [ ]:
gs_vary_reg = gs_df.loc[(gs_alpha_match & gs_iter_match & gs_rank_match),:]

In [ ]:
gs_vary_reg

In [ ]:
gs_vary_iter = gs_df.loc[(gs_alpha_match & gs_reg_match & gs_rank_match),:]

In [ ]:
gs_vary_iter

In [ ]:
gs_df.sort_values(['rmse']).head(10)

So quick inspection on these, lets:
- keep `rank` = 5
- When compared to all the other combos, the differences in `alpha`s seem to not really move the needle > 500. so let's just call it `1000`
- Keep `maxIter` at `20`; experience to date with my assets seems to show 20 is max capability before technical difficulties arise.
- Similar with `alpha`, the marginal change in error due to changing `reg` is really small. So let's just assume the default `.01`.

So, that means we are done selecting! We may really be pushing overfitting.

One last thing, let chart change in RMSE over change in alpha.

In [ ]:
alpha_graph_df = gs_vary_alpha.copy()

In [ ]:
alpha_graph_df['params_desc'] = (
                                '\u03B1=' + alpha_graph_df['alpha'].map(str) 
                                )
                                 

In [ ]:
alpha_graph_df

In [ ]:
sns.set(style="whitegrid")
sns.set(font_scale=2)

fig, ax = plt.subplots(figsize=(10, 10))

# Plot RMSE
sns.set_color_codes("pastel")

values = alpha_graph_df['params_desc'].tolist()

clrs = ['salmon' if (y == '\u03B1=1000') else 'steelblue' for y in values ]

s = sns.barplot(x="rmse", y="params_desc", data=alpha_graph_df,
                label="RMSE",
                palette=clrs)

# Add a legend and informative axis label

xlabel = "Max Iterations: " + str(best_max_iter) + " | Latent Factors: " + str(best_rank)
# xlabel


ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(ylabel="",
       xlabel=xlabel)
ax.set_title("Change in Error over Alpha")

sns.despine(left=True, bottom=True)
fig = s.get_figure()
# fig.savefig('support_data/alphas_20190905.png') 

---

---

---

---

---

---

One last thing, let's chart change in RMSE over change in alpha.

In [ ]:
alpha_graph_df = gs_vary_alpha.copy()

In [ ]:
alpha_graph_df['params_desc'] = (
                                '\u03B1=' + alpha_graph_df['alpha'].map(str) 
                                )
                                 

In [ ]:
alpha_graph_df

In [ ]:
sns.set(style="whitegrid")
sns.set(font_scale=2)

fig, ax = plt.subplots(figsize=(10, 10))

# Plot RMSE
sns.set_color_codes("pastel")

values = alpha_graph_df['params_desc'].tolist()

clrs = ['salmon' if (y == '\u03B1=1000') else 'steelblue' for y in values ]

s = sns.barplot(x="rmse", y="params_desc", data=alpha_graph_df,
                label="RMSE",
                palette=clrs)

# Add a legend and informative axis label

xlabel = "Max Iterations: " + str(best_max_iter) + " | Latent Factors: " + str(best_rank)
# xlabel


ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(ylabel="",
       xlabel=xlabel)
ax.set_title("Change in Error over Alpha")

sns.despine(left=True, bottom=True)
fig = s.get_figure()
fig.savefig('support_data/alphas_20190905.png') 

Note those that iterations = 35 is the max of our grid.

In [ ]:
# hyper-param config
num_iterations = [20, 25, 40]
ranks = [4,5]
reg_params = [0.01, 0.1]
alphas = [40, 50, 100]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
param_errs_rd_2 = params_errs

In [ ]:
with open('support_data/params_errs_20190907b.pkl', 'wb') as f:
    pickle.dump(param_errs_rd_2, f)
    
# Example - load picklea
# pickle_in = open("support_data/params_errs_rd1.pkl","rb")
# pe1 = pickle.load(pickle_in)

#### Use this to reload the Grid Search results

In [ ]:
pickle_in = open('support_data/params_errs_20190907b.pkl', 'rb')
params_errs_2 = pickle.load(pickle_in)
                         

In [ ]:
gs_cols = ['max_iters', 'reg', 'rank', 'alpha', 'rmse']

In [ ]:
gs_df = pd.DataFrame(params_errs_2, columns=gs_cols)

In [ ]:
gs_df.head()

In [ ]:
min_err = gs_df.rmse.min()

In [ ]:
min_df = gs_df.loc[gs_df['rmse']==min_err]

In [ ]:
min_df

In [ ]:
best_max_iter = min_df['max_iters'].iloc[0]
best_reg = min_df['reg'].iloc[0]
best_rank = min_df['rank'].iloc[0]
best_alpha = min_df['alpha'].iloc[0]

Let's do some visual comparisons.

In [ ]:
gs_rank_match = (gs_df['rank']==best_rank)
gs_reg_match = (gs_df['reg']==best_reg)
gs_iter_match = (gs_df['max_iters']==best_max_iter)
gs_alpha_match = (gs_df['alpha']==best_alpha)

In [ ]:
gs_vary_rank = gs_df.loc[(gs_reg_match & gs_iter_match & gs_alpha_match),:]

In [ ]:
gs_vary_rank

In [ ]:
gs_vary_alpha = gs_df.loc[(gs_reg_match & gs_iter_match & gs_rank_match),:]

In [ ]:
gs_vary_alpha

In [ ]:
gs_vary_reg = gs_df.loc[(gs_alpha_match & gs_iter_match & gs_rank_match),:]

In [ ]:
gs_vary_reg

In [ ]:
gs_vary_iter = gs_df.loc[(gs_alpha_match & gs_reg_match & gs_rank_match),:]

In [ ]:
gs_vary_iter

So quick inspection on these, lets:
- keep `rank` = 5
- When compared to all the other combos, the differences in `alpha`s seem to not really move the needle > 500. so let's just call it `1000`
- Keep `maxIter` at `20`; experience to date with my assets seems to show 20 is max capability before technical difficulties arise.
- Similar with `alpha`, the marginal change in error due to changing `reg` is really small. So let's just assume the default `.01`.

So, that means we are done selecting! We may really be pushing overfitting.

One last thing, let chart change in RMSE over change in alpha.

In [ ]:
alpha_graph_df = gs_vary_alpha.copy()

In [ ]:
alpha_graph_df['params_desc'] = (
                                '\u03B1=' + alpha_graph_df['alpha'].map(str) 
                                )
                                 

In [ ]:
alpha_graph_df

In [ ]:
sns.set(style="whitegrid")
sns.set(font_scale=2)

fig, ax = plt.subplots(figsize=(10, 10))

# Plot RMSE
sns.set_color_codes("pastel")

values = alpha_graph_df['params_desc'].tolist()

clrs = ['salmon' if (y == '\u03B1=1000') else 'steelblue' for y in values ]

s = sns.barplot(x="rmse", y="params_desc", data=alpha_graph_df,
                label="RMSE",
                palette=clrs)

# Add a legend and informative axis label

xlabel = "Max Iterations: " + str(best_max_iter) + " | Latent Factors: " + str(best_rank)
# xlabel


ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(ylabel="",
       xlabel=xlabel)
ax.set_title("Change in Error over Alpha")

sns.despine(left=True, bottom=True)
fig = s.get_figure()
#fig.savefig('support_data/alphas_20190905.png') 

In [ ]:
# hyper-param config
num_iterations = [20]
ranks = [3,4]
reg_params = [0.01]
alphas = [100, 500, 1000, 1500]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
# hyper-param config
num_iterations = [30, 50, 75, 100]
ranks = [5]
reg_params = [1, 5, 10]
alphas = [1000]

In [ ]:
# grid search and select best model
start_time = time.time()
final_model, params_errs = cr.train_ALS(gs_train, gs_val, eval_reg, 
                                        num_iterations, reg_params, 
                                        ranks, alphas)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

**OK**. Let's call it good. 

## Results 
Looks like the best parameters we could find are:
- `maxIter` = 20
- `rank` = 5
- `regParam` = 0.1 (default)
- `alpha` = 1000

Let's _-validate this candidate model.

## Cross Validation

Let's cross-validate because we didn't actually do it in the grid search. We want to make sure that the selected model is not overfitting.

The built-in cross validator in `Spark` keeps breaking when I try to use it, so let's build our own function.

In [ ]:
k = 10

In [ ]:
folds = cr.get_spark_k_folds(train, k=k, random_seed=random_seed)

In [ ]:
# Create ALS instance for cv with our chosen parametrs
als_cv = ALS(maxIter=best_max_iter,
          rank=best_rank,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          regParam=best_reg,
          alpha=best_alpha,
          coldStartStrategy='drop', # we want to drop so can get through CV
          seed=random_seed)

In [ ]:
errors = cr.get_cv_errors(folds, als_cv, eval_reg)

In [ ]:
# Make sure that # of errors = k
k == len(errors)

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(errors), np.std(errors) * 2))

Looks stable. Let's go with it.

## Test the Candidate Model

Test vs our holdout set.

In [ ]:
best_max_iter = 20
best_reg = 0.1
best_rank = 5
best_alpha = 1000

In [ ]:
# Create ALS instance and fit model
als = ALS(maxIter=best_max_iter,
          rank=best_rank,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          regParam=best_reg,
          alpha=best_alpha,
          coldStartStrategy='drop', # To get our eval
          seed=random_seed)
model_use = als.fit(train)

In [ ]:
# get predictions on test
test_preds = model_use.transform(test)

# Evaluate test
test_rmse = eval_reg.evaluate(test_preds)
test_rmse

Well, this is unexpected. Test error being noticeably lower than train error usually indicates an unknown fit. Since we trained on 'train' data we would expect test error to be at minimum as worse AND _probably_ a little worse than train. Not less than.

It's not THAT much better, but need to make note of it. For now we need to move on.

In [ ]:
# Create ALS instance and fit model
als = ALS(maxIter=best_max_iter,
          rank=best_rank,
          userCol='account_id',
          itemCol='comic_id',
          ratingCol='bought',
          implicitPrefs=True,
          regParam=best_reg,
          alpha=best_alpha,
          coldStartStrategy='nan', # To get our eval
          seed=random_seed)
model_use = als.fit(train)

#### Save the item factors for future use!

In [ ]:
item_factors = model_use.itemFactors.toPandas()

In [ ]:
item_factors.shape

In [ ]:
!ls

In [ ]:
item_factors.to_pickle("support_data/item_factors_20190916.pkl")

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
item_factors.head()

Test unpickle

In [ ]:
unpickled_items = pd.read_pickle('support_data/item_factors_20190916.pkl')

### Get Top N recommendations for Single User

Let's make a reference list of `account_id`'s, for testing purposes.

In [ ]:
n_to_test = 2

users = (sold.select(als.getUserCol())
                          .sample(False
                                  ,n_to_test/sold.count()
                                  )
        )
users.persist()
users.show(2)

We developed and wrote the functionality out to a function in `comic_recs.py`

###  Testing function!

- Pass the function to a pandas dataframe. 
- Function will ask for an account_id.
- Will return top n, n defined in parameters.

In [ ]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=5)
top_n_df

In [ ]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=5)
top_n_df

In [ ]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=model_use, topn=10)
top_n_df

## Conclusions
- Seems realistic? Only three tests, but the results seem 'individualized' in the sense that there is no overlap between the sets (albeit small samples).

## Save the Model!

In [ ]:
model_use.save('models/als_use_20190916')

## Retrieving Saved Model

In [ ]:
comic_rec_model = ALSModel.load('models/als_use_20190916')

In [ ]:
top_n_df = cr.get_top_n_new_recs(spark=spark, model=comic_rec_model, topn=10)
top_n_df